# Ring Throw Parameter Sweeping

This notebook demonstrates how to:
1. Load the robotics environment with RAI.
2. Parameterize certain variables in the ring-throw code.
3. Loop over a range of values for those parameters.
4. Run the simulation for each combination.
5. Log the outcomes to a CSV.


In [ ]:
import robotic as ry
import numpy as np
import csv
import os

# Ensure RAI can render if on a local/GUI system:
os.environ['DISPLAY'] = ':0'

# 1) Load Environment
C = ry.Config()
C.addFile("/home/yagz/rai_venv/robotics/penvironment.g")  # Adjust path if needed
C.view(False)

# Create a BotOp to interact with the environment
bot = ry.BotOp(C, useRealRobot=False)
bot.home(C)

def align_gripper_to_ring():
    approach_waypoint = C.addFrame('approach_waypoint', 'ring1')
    align_waypoint = C.addFrame('align_waypoint', 'ring1')
    approach_waypoint.setShape(ry.ST.marker, size=[.1])
    approach_waypoint.setRelativePose('t(0 0 0.1) d(90 0 0 0)')

    align_waypoint.setShape(ry.ST.marker, size=[.1])
    align_waypoint.setRelativePose(' d(90 0 0 0)')

    komo = ry.KOMO()
    komo.setConfig(C, True)
    komo.setTiming(2., 1, 5., 0)
    komo.addControlObjective([], 0, 1e-0)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e1])
    komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)
    komo.addObjective([1.], ry.FS.positionDiff, ['l_gripper', 'approach_waypoint'], ry.OT.eq, [1e2])
    komo.addObjective([2.], ry.FS.positionDiff, ['l_gripper', 'align_waypoint'], ry.OT.eq, [1e2])

    solver = ry.NLP_Solver()
    solver.setProblem(komo.nlp())
    solver.setOptions(stopTolerance=1e-2, verbose=4)
    solver.solve()

    return komo.getPath()

def grasp_ring():
    bot.gripperCloseGrasp(ry._left, 'ring1', force=10.0, width=0, speed=1)
    while not bot.gripperDone(ry._left):
        bot.sync(C, 0.1)
    print("Ring successfully grasped by the gripper.")

def throw():
    rel_point = [0, 1.6, 1]
    komo = ry.KOMO()
    komo.setConfig(C, True)
    komo.setTiming(2., 1, 5., 1)
    komo.addControlObjective([], 0, 1e-0)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e1])
    komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)

    # Position objectives
    komo.addObjective([1.], ry.FS.position, ['l_gripper'], ry.OT.eq, [1e1], target=rel_point )
    # Velocity objective (order=1)
    komo.addObjective([1.], ry.FS.position, ['l_gripper'], ry.OT.eq, [1e1],
                      target = [0, -3.4, 8.355], order=1)

    solver = ry.NLP_Solver()
    solver.setProblem(komo.nlp())
    solver.setOptions(stopTolerance=1e-2, verbose=4)
    solver.solve()
    return komo.getPath()

def is_ring_on_pole():
    """
    Placeholder for success detection logic:
    For example, measure distance between ring frame and some 'pole' frame.
    Return True if ring is close enough to the pole.
    """
    # TODO: Implement real logic.  For now, always False.
    return False

def run_ring_throw(q2_val, time1, time2, release_threshold):
    """
    Executes the ring throw with the given parameters:
      q2_val => the offset for q[2]
      time1, time2 => the timing for bot.move(throw_path, [time1, time2])
      release_threshold => the Z-height at which we open the gripper

    Returns True if success, else False.
    """

    # 1) Align
    grasp_path = align_gripper_to_ring()
    bot.move(grasp_path, [1.0, 1.5])  # fixed for alignment
    bot.wait(C, forKeyPressed=False, forTimeToEnd=True)

    # 2) Grasp
    grasp_ring()
    bot.wait(C, forKeyPressed=False, forTimeToEnd=True)

    # 3) Modify q[2]
    q = bot.get_qHome()
    # q[2] is between -0.51 and -0.11
    q[2] = q2_val
    # keep q[3] offset as originally code does ( +4 )
    q[3] = q[3] + 4
    bot.moveTo(q)
    bot.wait(C, forKeyPressed=False, forTimeToEnd=True)

    # 4) Throw path
    throw_path = throw()
    # time1 in [0..0.2], time2 in [1..1.5]
    bot.move(throw_path, [time1, time2])

    # 5) Release logic: if gripper_height > release_threshold => open gripper
    while bot.getTimeToEnd() > 0:
        bot.sync(C, 0.05)
        gripper_position = C.frame("l_gripper").getPosition()
        gripper_height = gripper_position[2]

        if gripper_height > release_threshold:
            bot.gripperMove(ry._left, width=10, speed=10)
    bot.wait(C, forKeyPressed=False, forTimeToEnd=True)

    # 6) Evaluate success
    success = is_ring_on_pole()
    return success


## 2) Parameter Sweeping (Runner)

We will define the ranges for:
- **`q2_val`** in `[-0.51..-0.11]`
- **`time1`** in `[0..0.2]`
- **`time2`** in `[1..1.5]`
- **`release_threshold`** in `[0.4..0.7]`

Then we will loop over them, run the sim, and record the results in a CSV.

In [ ]:
import csv

# Let's define some step sizes for each parameter
q2_vals = np.linspace(-0.51, -0.11, 5)           # e.g. 5 steps
time1_vals = np.linspace(0.0, 0.2, 3)            # e.g. 0.0, 0.1, 0.2
time2_vals = np.linspace(1.0, 1.5, 3)            # e.g. 1.0, 1.25, 1.5
release_threshes = np.linspace(0.4, 0.7, 4)      # e.g. 0.4, 0.5, 0.6, 0.7

output_file = "throw_results.csv"

with open(output_file, "w", newline="") as f:
    writer = csv.writer(f)
    # CSV header
    writer.writerow(["q2_val", "time1", "time2", "release_threshold", "success"])

    # nested loops
    for q2 in q2_vals:
        for t1 in time1_vals:
            for t2 in time2_vals:
                for rel_th in release_threshes:
                    print(f"Running with q2={q2:.2f}, t1={t1:.2f}, t2={t2:.2f}, rel_th={rel_th:.2f}")

                    success = run_ring_throw(
                        q2_val = q2,
                        time1 = t1,
                        time2 = t2,
                        release_threshold = rel_th
                    )

                    writer.writerow([q2, t1, t2, rel_th, success])

print("\nAll simulations complete. Results saved in", output_file)
